In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os                                              
import json
import numpy as np
from tensorflow.keras.models import load_model
import cv2

In [ ]:
# === Settings ===
image_size = (128, 128)
batch_size = 2
epochs = 100
num_classes = 7
dataset_dir = "theDataset1"

# === Load datasets ===
train_datagen = ImageDataGenerator(rescale=1.0/255)
val_datagen   = ImageDataGenerator(rescale=1.0/255)
test_datagen  = ImageDataGenerator(rescale=1.0/255)

train_gen = train_datagen.flow_from_directory(
    os.path.join(dataset_dir, "Train"),
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical"  # since you're using softmax and 7 classes
)

print(train_gen.class_indices)
# {'angle_1': 0, 'angle_2': 1, ..., 'angle_6': 5, 'noWave': 6}

from collections import Counter

# === Step 1: Compute class counts
counter = Counter(train_gen.classes)
total = sum(counter.values())

# === Step 2: Initial inverse-frequency weighting
class_weight = {i: total / (len(counter) * count) for i, count in counter.items()}

# === Step 3: Boost wave classes
wave_boost_factor = 1.0
wave_class_indices = [0, 1, 2, 3, 4, 5]  # angle_1 to angle_6
for i in wave_class_indices:
    class_weight[i] *= wave_boost_factor

print("📊 Final class weights:", class_weight)


val_gen = val_datagen.flow_from_directory(
    os.path.join(dataset_dir, "Validation"),
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical"
)

test_gen = test_datagen.flow_from_directory(
    os.path.join(dataset_dir, "Test"),
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False  # keep test order stable
)






class FullDatasetPredictionLogger(tf.keras.callbacks.Callback):
    def __init__(self, train_gen, val_gen, test_gen, log_dir="PredictionLogs"):
        super().__init__()
        self.train_gen = train_gen
        self.val_gen = val_gen
        self.test_gen = test_gen
        self.log_dir = log_dir
        os.makedirs(log_dir, exist_ok=True)

        self.logs = {
            "Train": [],
            "Validation": [],
            "Test": []
        }

    def _log_predictions_for_dataset(self, dataset_name, generator, epoch):
        # Predict for the entire dataset
        probs = self.model.predict(generator, verbose=0)
        true_labels = np.argmax(np.vstack([generator[i][1] for i in range(len(generator))]), axis=1)

        self.logs[dataset_name].append({
            "epoch": epoch + 1,
            "predictions": [
                {
                    "true_label": int(true_labels[i]),
                    "probs": probs[i].tolist()
                }
                for i in range(len(probs))
            ]
        })

        # Save to disk
        with open(os.path.join(self.log_dir, f"{dataset_name}_predictions.json"), 'w') as f:
            json.dump(self.logs[dataset_name], f, indent=2)

    def on_epoch_end(self, epoch, logs=None):
        self._log_predictions_for_dataset("Train", self.train_gen, epoch)
        self._log_predictions_for_dataset("Validation", self.val_gen, epoch)
        self._log_predictions_for_dataset("Test", self.test_gen, epoch)

     

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback

es = EarlyStopping(monitor = "val_accuracy", min_delta = 0.01, patience = 10, verbose = 1)
class CustomModelCheckpoint(Callback):
    def __init__(self, filepath, save_freq):
        super(CustomModelCheckpoint, self).__init__()
        self.filepath = filepath
        self.save_freq = save_freq
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:  # Save on specific iterations (1-indexed)
            self.model.save(self.filepath.format(epoch=epoch + 1))

model_cp = ModelCheckpoint(filepath = 'Dgesture_Best.keras', monitor = "val_accuracy",
                           save_best_only = True,
                           save_weights_only = False,
                           verbose = 1)
# Define your custom checkpoint for specific iterations
specific_iteration_cp = CustomModelCheckpoint(filepath='Keras/Dgesture_Epoch{epoch:02d}.keras',
                                              save_freq=10)  

class HistorySaver(tf.keras.callbacks.Callback):
    def __init__(self, file_path):
        super().__init__()
        self.file_path = file_path
        self.history = []
        self.last_epoch = 0

        # Load existing history if file exists
        if os.path.exists(self.file_path):
            with open(self.file_path, 'r') as file:
                data = json.load(file)
                self.history = data.get('history', [])
                self.last_epoch = data.get('last_epoch', 0)

class SaveHistoryAnd90Acc(Callback):
    def __init__(self, history_path='training_history.json', save_path='gesture_90acc.keras', acc_threshold=0.90):
        super().__init__()
        self.history_path = history_path
        self.save_path = save_path
        self.acc_threshold = acc_threshold
        self.history = []
        self.last_epoch = 0
        self.saved = False  # Tracks if model was already saved at threshold

        # Load existing history if it exists
        if os.path.exists(self.history_path):
            with open(self.history_path, 'r') as file:
                data = json.load(file)
                self.history = data.get('history', [])
                self.last_epoch = data.get('last_epoch', 0)
                
    def on_epoch_end(self, epoch, logs=None):
        # Save training history
        self.history.append({**logs, 'epoch': epoch + 1})
        with open(self.history_path, 'w') as file:
            json.dump({
                'history': self.history,
                'last_epoch': epoch + 1
            }, file, indent=4)

        # Check for 90% training accuracy
        acc = logs.get('accuracy')
        if acc is not None and not self.saved and acc >= self.acc_threshold:
            self.model.save(self.save_path)
            print(f"\n📦 Saved model at epoch {epoch+1} (Training Accuracy: {acc:.4f}) → {self.save_path}")
            self.saved = True
                

class SaveHistoryAnd80Acc(Callback):
    def __init__(self, history_path='training_history.json', save_path='gesture_80acc.keras', acc_threshold=0.80):
        super().__init__()
        self.history_path = history_path
        self.save_path = save_path
        self.acc_threshold = acc_threshold
        self.history = []
        self.last_epoch = 0
        self.saved = False  # Tracks if model was already saved at threshold

        # Load existing history if it exists
        if os.path.exists(self.history_path):
            with open(self.history_path, 'r') as file:
                data = json.load(file)
                self.history = data.get('history', [])
                self.last_epoch = data.get('last_epoch', 0)

    def on_epoch_end(self, epoch, logs=None):
        # Save training history
        self.history.append({**logs, 'epoch': epoch + 1})
        with open(self.history_path, 'w') as file:
            json.dump({
                'history': self.history,
                'last_epoch': epoch + 1
            }, file, indent=4)

        # Check for 80% training accuracy
        acc = logs.get('accuracy')
        if acc is not None and not self.saved and acc >= self.acc_threshold:
            self.model.save(self.save_path)
            print(f"\n📦 Saved model at epoch {epoch+1} (Training Accuracy: {acc:.4f}) → {self.save_path}")
            self.saved = True
                
class SaveHistoryAnd80Acc(Callback):
    def __init__(self, history_path='training_history.json', save_path='gesture_80acc.keras', acc_threshold=0.80):
        super().__init__()
        self.history_path = history_path
        self.save_path = save_path
        self.acc_threshold = acc_threshold
        self.history = []
        self.last_epoch = 0
        self.saved = False  # Tracks if model was already saved at threshold

        # Load existing history if it exists
        if os.path.exists(self.history_path):
            with open(self.history_path, 'r') as file:
                data = json.load(file)
                self.history = data.get('history', [])
                self.last_epoch = data.get('last_epoch', 0)
    
    def on_epoch_end(self, epoch, logs=None):
        # Save training history
        self.history.append({**logs, 'epoch': epoch + 1})
        with open(self.history_path, 'w') as file:
            json.dump({
                'history': self.history,
                'last_epoch': epoch + 1
            }, file, indent=4)

        # Check for 80% training accuracy
        acc = logs.get('accuracy')
        if acc is not None and not self.saved and acc >= self.acc_threshold:
            self.model.save(self.save_path)
            print(f"\n📦 Saved model at epoch {epoch+1} (Training Accuracy: {acc:.4f}) → {self.save_path}")
            self.saved = True

class SaveHistoryAnd70Acc(Callback):
    def __init__(self, history_path='training_history.json', save_path='gesture_70acc.keras', acc_threshold=0.70):
        super().__init__()
        self.history_path = history_path
        self.save_path = save_path
        self.acc_threshold = acc_threshold
        self.history = []
        self.last_epoch = 0
        self.saved = False  # Tracks if model was already saved at threshold

        # Load existing history if it exists
        if os.path.exists(self.history_path):
            with open(self.history_path, 'r') as file:
                data = json.load(file)
                self.history = data.get('history', [])
                self.last_epoch = data.get('last_epoch', 0)
    
    def on_epoch_end(self, epoch, logs=None):
        # Save training history
        self.history.append({**logs, 'epoch': epoch + 1})
        with open(self.history_path, 'w') as file:
            json.dump({
                'history': self.history,
                'last_epoch': epoch + 1
            }, file, indent=4)

        # Check for 70% training accuracy
        acc = logs.get('accuracy')
        if acc is not None and not self.saved and acc >= self.acc_threshold:
            self.model.save(self.save_path)
            print(f"\n📦 Saved model at epoch {epoch+1} (Training Accuracy: {acc:.4f}) → {self.save_path}")
            self.saved = True
   
combined_callback = SaveHistoryAnd90Acc(
    history_path='trainingHistory.json',
    save_path='Dgesture_90acc.keras',
    acc_threshold=0.90
)

combined_callback1 = SaveHistoryAnd80Acc(
    history_path='trainingHistory.json',
    save_path='Dgesture_80acc.keras',
    acc_threshold=0.80
)

combined_callback2 = SaveHistoryAnd80Acc(
    history_path='trainingHistory.json',
    save_path='Dgesture_70acc.keras',
    acc_threshold=0.70
)
# Usage
file_path = 'trainingHistory.json'
history_saver = HistorySaver(file_path)
# Load the last completed epoch to start from there
start_epoch = history_saver.last_epoch


prediction_logger = FullDatasetPredictionLogger(train_gen, val_gen, test_gen)
callbacks = [model_cp, specific_iteration_cp, history_saver, prediction_logger,combined_callback, combined_callback1, combined_callback2]


In [ ]:
model = load_model("Keras/C1gesture_Epoch90.keras")
model.summary()
# === Compile the model ===
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    class_weight=class_weight,
    callbacks=callbacks,
    verbose=1
)

# === Evaluate on test set ===
test_loss, test_acc = model.evaluate(test_gen, verbose=1)
print(f"\n📊 Final Test Accuracy: {test_acc:.4f}")

# === Save model ===
model.save("gesture_CNN.keras")
print("✅ Model saved as gesture_CNN.keras")
